## Do not use - do not run

bits of code i wrote and labeled as "innefficient", "useless", or "do not run" as i wrote this comments a few days ago i totally forgot if i use thoses or not tbh

In [ ]:
## do not use - innefficient
def printAllTables(gt_table=gt_table, extracted_table=extracted_table):
    '''affiche le contenue toutes les tables de la base de données.'''

    conn = sqlite3.connect('content/database.db')
    cursor = conn.cursor()

    # testing if the data is inserted correctly
    cursor.execute(f"SELECT * FROM {gt_table} LIMIT 15;")
    print("Ground Truth Table:")
    for row in cursor.fetchall():
        print(row)

    cursor.execute(f"SELECT * FROM {extracted_table} ;")
    print("\nExtracted Table:")
    for row in cursor.fetchall():
        print(row)

    conn.commit()
    conn.close()

def countElements():
    '''counts the number of elements in each table and prints the result. -- specific to our database.'''

    conn = sqlite3.connect('content/database.db')
    cursor = conn.cursor()

    # testing if the data is inserted correctly
    cursor.execute(f"SELECT COUNT(DISTINCT Study) FROM {gt_table};")
    print("Ground Truth Table count:")
    print(cursor.fetchall()[0])
        

    cursor.execute(f"SELECT COUNT(DISTINCT Study) FROM {extracted_table} ;")
    print("\nExtracted Table count:")
    print(cursor.fetchall()[0])

    cursor.execute(f"""
        SELECT COUNT(ext.Study)
        FROM {gt_table} AS gt
                JOIN {extracted_table} AS ext ON gt.Study = ext.Study
        """)
    print("Total matched rows:", cursor.fetchone()[0])


    conn.commit()
    conn.close()

#countElements()
#printAllTables()

In [ ]:
### still doesn't work yet 
# TODO : debug function creatingBD_multiSplit
def creatingBD_multiSplit(ext_splitted, ground_truth_df):
    """
    Creates a SQLite database with the same structure as the given DataFrame and inserts the data into it.
    Args:
        ext_splitted (list): List of dictionaries containing extracted data.
        ground_truth_df (DataFrame): The ground truth DataFrame.
    """
    # Connect to database
    conn = sqlite3.connect('content/database.db')

    # Dropping the tables if they exist
    conn.execute("DROP TABLE IF EXISTS GroundTruth")
    for ext in ext_splitted:
        conn.execute(f"DROP TABLE IF EXISTS {ext['dir'].replace('extracted_', '')}")

    # Create GroundTruth table
    ground_truth_df.to_sql('GroundTruth', conn, if_exists='append', index=False)

    # Create ExtractedData table for each extracted DataFrame
    for ext in ext_splitted:
        ext['data'].to_sql({ext['dir'].replace('extracted_', '')}, conn, if_exists='append', index=False)
        print(f"Table {ext['dir']} table got created.")

    print("Tables GroundTruth and ExtractedData created with the same structure as the CSV and data inserted.")

    # Done
    conn.commit()
    conn.close()

def inserting_a_single_table_in_DB(conn, df):
    """inserting a single table at a time in the database
        args:
            conn (sqlite3.Connection): The SQLite connection object.
            df (DataFrame): The DataFrame to be inserted into the database.
    """
    #connecting to the database
    conn = sqlite3.connect('content/database.db')

    # Dropping the table if it exists
    conn.execute(f"DROP TABLE IF EXISTS {df[dir].replace('extracted_', '')}")

    # Populating the table with the DataFrame
    df['data'].to_sql(df[dir].replace('extracted_', ''), conn, if_exists='append', index=False)
    print(f"Table {df[dir]} got created.")

    # Done
    conn.commit()
    conn.close()
